Продемонстрируем, как свертки работают на практике, на численном примере в TensorFlow. Начнем, как обычно, с импортирования TensorFlow и numpy:

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\Art\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Поскольку мы никакую модель обучать не планируем, в качестве переменных будет достаточно определить «заглушки» заданного размера:

In [2]:
x_inp = tf.placeholder(tf.float32, [5, 5])
w_inp = tf.placeholder(tf.float32, [3, 3])

Итак, операция свертки в TensorFlow оперирует четырехмерными тензорами, а не обычными матрицами, поэтому входные данные нужно привести к соответствующим размерностям:

In [3]:
x = tf.reshape(x_inp, [1, 5, 5, 1])
w = tf.reshape(w_inp, [3, 3, 1, 1])

Напомним, что первая размерность тензора х обозначает количество изображений в мини-батче, а последняя — число каналов изображения. Так как мы всего лишь хотим проиллюстрировать описанную выше теорию, нам будет достаточно одной черно-белой картинки. Теперь можно задавать саму операцию свертки:

In [4]:
x_valid = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="VALID")
x_same = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="SAME")
x_valid_half= tf.nn.conv2d(x, w, strides=[1, 2, 2, 1], padding="VALID")
x_same_half= tf.nn.conv2d(x, w, strides=[1, 2, 2, 1], padding="SAME")

Аргумент strides задает шаг по изображению. Обратите внимание, что параметр strides просто определяет, как часто мы применяем фильтры по каждой размерности входного тензора, а размерностей этих у него в данном случае четыре. Поэтому, например, первая компонента strides соответствует
разным примерам в мини-батче, и если бы она была не равна единице, мы бы просто пропускали часть входных примеров. В данном случае это выглядит странно, и для функции tf.nn.conv2d вряд ли есть смысл задавать strides с неединичной первой компонентой (или пропускать часть цветовых фильтров в четвертой компоненте), но аргумент strides в TensorFlow является более общим и может быть применен к любому тензору, отсюда и «лишние» размерности.  

В нашем примере, меняя параметр strides, мы будем просто получать некоторые подматрицы тех матриц, которые мы вычисляли выше. Например, для strides = [l, 2, 2, 1] мы будем пропускать каждую вторую размерность и по строкам, и по столбцам.  

Итак, наша «модель» задана. Запишем входные данные:

In [5]:
x = np.array([[0, 1, 2, 1, 0],
             [4, 1, 0, 1, 0],
             [2, 0, 1, 1, 1],
             [1, 2, 3, 1, 0],
             [0, 4, 3, 2, 0]])
w = np.array([[0, 1, 0],
             [1, 0, 1],
             [2, 1, 0]])

Осталось только объявить сессию и вычислить результат:

In [6]:
sess = tf.Session()
y_valid, y_same, y_valid_half, y_same_half = sess.run(
    [x_valid, x_same, x_valid_half, x_same_half],
    feed_dict={x_inp: x, w_inp: w}
)

Давайте для проверки выведем то, что у нас получилось; поскольку на выходе у нас по-прежнему будут получаться четырехмерные тензоры, а мы хотели бы увидеть обыкновенные матрицы, некоторые (тривиальные) размерности мы зафиксируем нулями:

In [7]:
print("padding=VALID:\n", y_valid[0, :, :, 0])
print("padding=SAME:\n", y_same[0, :, :, 0])
print("padding=VALID, stride 2:\n", y_valid_half[0, :, :, 0])
print("padding=SAME, stride 2:\n", y_same_half[0, :, :, 0])

padding=VALID:
 [[ 9.  5.  4.]
 [ 8.  8. 10.]
 [ 8. 15. 12.]]
padding=SAME:
 [[ 5. 11.  4.  3.  3.]
 [ 3.  9.  5.  4.  4.]
 [ 5.  8.  8. 10.  3.]
 [ 4.  8. 15. 12.  6.]
 [ 5.  5.  9.  4.  2.]]
padding=VALID, stride 2:
 [[ 9.  4.]
 [ 8. 12.]]
padding=SAME, stride 2:
 [[5. 4. 3.]
 [5. 8. 3.]
 [5. 9. 2.]]


Итак, мы научились делать операцию свертки. После нее можно, как мы уже говорили, применить ту или иную нелинейную функцию h: она будет просто применяться к каждому элементу полученного тензора по отдельности. Но это еще не все. В классическом сверточном слое, кроме линейной свертки и следующей за ней нелинейности, есть и еще одна операция: субдискретизация (pooling; по-русски ее иногда называют еще операцией «подвыборки», от альтернативного английского термина subsampling).  

Смысл субдискретизации прост: в сверточных сетях обычно исходят из предположения, что наличие или отсутствие того или иного признака гораздо важнее, чем его точные координаты. Например, при распознавании лиц сверточной сетью нам гораздо важнее понять, есть ли на фотографии лицо и чье, чем узнать, с какого конкретно пиксела оно начинается и в каком заканчивается. Поэтому можно позволить себе «обобщить» выделяемые признаки, потеряв часть информации об их местоположении, но зато сократив размерность.  

Обычно в качестве операции субдискретизации к каждой локальной группе нейронов применяется операция взятия максимума (max-pooling).  

Хотя в результате субдискретизации действительно теряется часть информации, сеть становится более устойчивой к небольшим трансформациям изображения вроде сдвига или поворота.  

Чтобы сделать все это в TensorFlow, как и в случае с операцией свертки, сначала зададим заглушку для входного «изображения» и приведем ее к нужной размерности:

In [8]:
x_inp = tf.placeholder(tf.float32, [4, 4])
x = tf.reshape(x_inp, [1, 4, 4, 1])

Теперь можно определять операции субдискретизации с помощью специальной функции tf.nn.max_pool; мы попробуем размеры шага 1 и 2:

In [9]:
x_valid = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="VALID")
x_valid_half = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

Обратите внимание, что здесь появился не только аргумент strides, который мы уже обсуждали, но и отдельно задаваемый размер окна ksize. Он тоже представляет собой четырехмерный тензор, то есть субдискретизацию можно проводить по любым размерностям, включая элементы мини-батчей и каналы. Это может опять показаться несколько странным; например, что может означать субдискретизация по каналам? Предположим, что перед нами стоит задача проверить, есть ли на данной фотографии попугай особого RGB-семейства, представители которого бывают только монохромными: идеального красного, зеленого или синего цвета. В таком случае мы можем решить, что есть смысл использовать субдискретизацию не только на пространственных измерениях, но и на каналах изображения, например для того, чтобы нейронная сеть могла как можно раньше избавиться от заведомо нерелевантных цветовых каналов. Звучит странно, но давайте попробуем теперь предположить, что на входе не отдельные фотографии, а видео в виде последовательных кадров. Тогда субдискретизация по каналам (если кадры соответствуют каналам) или мини-батчам (если входным примерам) внезапно становится очень осмысленной: соседние кадры почти всегда очень похожи, и если наша цель — распознать присутствие каких-то объектов в видео, то большую часть кадров можно спокойно выбросить.  

Доведем наш эксперимент до конца. Для этого объявим входную матрицу, произведем вычисления и выпишем результат:

In [10]:
x = np.array([[0, 1, 2, 1],
             [4, 1, 0, 1],
             [2, 0, 1, 1],
             [1, 2, 3, 1]])

y_valid, y_valid_half = sess.run(
    [x_valid, x_valid_half],
    feed_dict={x_inp: x}
)

In [11]:
print("padding=VALID:\n", y_valid[0, :, :, 0])
print("padding=VALID, stride 2:\n", y_valid_half[0, :, :, 0])

padding=VALID:
 [[4. 2. 2.]
 [4. 1. 1.]
 [2. 3. 3.]]
padding=VALID, stride 2:
 [[4. 2.]
 [2. 3.]]
